# ENTSO-E Daten

Die Daten der European Network of Transmission System Operators for Electricity (ENTSO-E) werden auf dem Transparenzportal veröffentlicht.

Über eine API lassen sich die Daten, sowie Historiendaten abfragen.

Diese werden mithilfe von Python analysiert und visualisiert.

Durch Plotly wurde unter https://nicht.datensch.eu/entsoe ein Dashboard der Daten erstellt.

ähnliche unabhängige Projekte:\
https://energy-charts.info/ \
https://www.electricitymap.org/map

<h1 id="tocheading">Inhaltsverzeichnis</h1>
<div id="toc"></div>
<script src="http://ajax.googleapis.com/ajax/libs/jquery/1.7.1/jquery.min.js" type="text/javascript"></script>
<script src="https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js"></script>

In [ ]:
%%javascript
//erzeugt Inhaltsverzeichnis
//https://github.com/kmahelona/ipython_notebook_goodies
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Begriffserklärungen / Glossar

* **Generation** beschreibt das erzeugte, also den gesamten eingespeisten Strom, ohne absorbierter Energie
* **Load/Last** beschreibt "das Verbrauchte", also das Erzeugte und Importierte minus absorbiertem und exportiertem.

die aktuellen Werte werden i.d.R. über ein 15 Minuten Mittel erhalten. Manche Länder senden jedoch nur stündlich ihre Daten.
Die Energie lässt sich dadurch durch Leistung * Zeit berechnen.

Eine Umrechnung in kWh bedeutet also ein Teilen durch Faktor 4 der Summe (wenn Stunde die Gruppierung ist)

## Bedienung der interaktiven Grafiken

Die Plots wurden mit Plotly erstellt.

* Diese Bibliothek erlaubt es durch anklicken in der Legende eine Datenreihe auszublenden.
* Durch ein Doppelklick auf eine Datenreihe kann eine Datenreihe isoliert betrachtet werden. Anschließend können weitere zu betrachtene Datenreihen ausgewählt werden
* Durch ziehen eines Intervalls, kann man einen genaueren Betrachtungsraum auswählen. Ein Doppelklick in das Diagramm springt wieder zum Ursprungsbereich

# API-Analysen, Erklärung der grundlegenden Funktionen und Erkenntnisse

## Daten abfragen und speichern

Die Daten stehen über eine API zur Verfügung. Hierzu benötigt man einen api_key, welchen man mit einer Mail-Adresse beantragen kann.
Anschließend kann man anfragen an die API stellen. Hierzu benötigt man für jede Area den zugehörigen Code.
Diese antwortet mit CSV bzw manchmal gezipptem CSV.
Die API findet sich unter dem Endpunkt: https://transparency.entsoe.eu/api

Der Umgang hiermit wird erheblich durch eine bestehende Python-Bibliothek erleichtert.

Mit der entsoe-py Bibliothek können wir die Daten direkt als Pandas-DataFrame runterladen.
Diese stellt die folgenden Endpunkte bereit:

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import matplotlib.pyplot as plt
client = EntsoePandasClient(api_key='ae2ed060-c25c-4eea-8ae4-007712f95375')

object_methods = [method_name for method_name in dir(client)
                  if callable(getattr(client, method_name))]
# only public ones
list(filter(lambda k: not str.startswith(k,'_'), object_methods))

wir beschränken uns hierbei auf:
* die Auslastung: `query_load`,
* die generierte Energie: `query_generation`,
* die generierte Energie pro Kraftwerk: `query_generation_per_plant`
* die vorhandenen Kapazitäten: `query_installed_generation_capacity`
* diese gibt es auch noch pro Kraftwerk: `query_installed_generation_capacity_per_unit`
* außerdem gibt es noch die Übertragungsdaten zwischen den Ländern: `query_crossborder_flows`

Der Beginn der Datenaufzeichnungen im neuen Format der ENTSO-E ist der 1.1.2015 für Deutschland. Andere Länder haben erst später mit den Aufzeichnungen angefangen.

In [ ]:
# german data available since
begin = pd.Timestamp('20150101', tz='Europe/Berlin')

Schauen wir uns mal die Auslastung der letzten 5 Tage an

In [ ]:
from datetime import datetime, date, timedelta
from entsoe.mappings import PSRTYPE_MAPPINGS,NEIGHBOURS,Area
areas = pd.DataFrame([[e.name,e.value,e._tz,e._meaning] for e in Area])
load = pd.DataFrame()
country_code = 'DE'

today=datetime.strftime(datetime.now().date(),'%Y%m%d')
prev5=datetime.now().date()-timedelta(days=5)
prev5str=datetime.strftime(prev5,'%Y%m%d')

start = pd.Timestamp(prev5str, tz='Europe/Berlin')
end = pd.Timestamp(today, tz='Europe/Berlin')

load[country_code] = client.query_load(country_code, start=start,end=end)


In [ ]:
df = pd.DataFrame(load['DE'])

import plotly.express as px 
fig = px.line(df, x=df.index, y='DE', title='Load for Germany')
fig.update_layout(
    title="Netzlast der letzten 5 Tage in DE",
    xaxis_title="Zeit",
    yaxis_title="Leistung in MW")
fig.show()

Zu beachten ist hierbei, dass die Daten zum einen pro Land erfasst werden, zum anderen auch pro BiddingZone, also pro Marktgebiet.
So gibt es ebenfalls Daten für DE, DE_LU, DE_AT_LU, DE_50HERTZ, DE_AMPRION.

Die Unterscheidung muss für alle Datenwerte beachtet werden, was visualisiert werden soll.

Intern gibt es für diese Bereiche noch eine nicht lesbare ID. Die Übersetzung in die Marktgebiete wie sie auf der Webseite zu finden sind, erfolgt bereits in dem Python-Package

In [ ]:
for e in Area:
    print(e.name)

Bzw sind hier zu finden:
https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_areas

## Karte der Kraftwerke

Die Kraftwerke eines Marktgebietes kann man sich über die API runterladen:

In [ ]:
def getProductionData():
    prod = client.query_installed_generation_capacity_per_unit('DE_AT_LU',start=start,end=end)
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('BE',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_AMPRION',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_50HZ',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_LU',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('FR',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('CH',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('AT',start=start,end=end)])
    prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('GB',start=start,end=end)])
    prod.index.name ='eic_code'
    return prod

%time prod = getProductionData()
prod.head()

Die erste Spalte ist hierbei der Energy-Identification-Code (EIC_CODE)

Siehe https://www.entsoe.eu/data/energy-identification-codes-eic/eic-approved-codes/

Allerdings sind hier keine Koordinaten enthalten. Die Koordinaten für "Konventionelle Kraftwerke" werden glücklicherweise von einer externen Webseite bereitgestellt:

Leider konnte ich keine Quelle finden, welche die Werte frei verfügbar nach EI_CODE aufgeschlüsselt für Solar und Wind ebenfalls enthält.

In [ ]:
df = pd.read_csv('https://data.open-power-system-data.org/conventional_power_plants/latest/conventional_power_plants_EU.csv')
# uns interessieren nur Daten, welche auch Koordinaten und einen Key enthalten
df.dropna(axis=0,subset=['lon','lat','eic_code'],inplace=True)
df.index = df['eic_code']

Nun können wir durch einen join der Datasets eine Karte von Kraftwerken erstellen:

In [ ]:
joined = prod.join(df, on='eic_code', how='inner')[['Name','lon','lat','capacity','Production Type','country']]
#joined.dropna(axis=0,subset=['lon','lat'],inplace=True)

newDf = prod.drop(df['eic_code'], errors='ignore').reset_index(drop=False)
print(prod.shape)
print(joined.shape)

prod['eic']=prod.index
p = pd.concat([prod['eic'],df['eic_code']],axis=0)
p =p.drop_duplicates(keep=False)

In [ ]:
pd.set_option('display.max_rows', 1000)
joined2 = prod.join(df, on='eic_code', how='outer')[['Name','lon','lat','capacity','Production Type','country']]

Die geringsten Kapazitäten haben hierbei Schweizer-Wasserspeicher.

In [ ]:
joined2.sort_values('capacity')[:50]

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(joined, lat="lat", lon="lon", color='country',hover_name="Name",hover_data=["capacity",'Production Type'],zoom=3)
fig.update_layout(title="Kraftwerkskarte, konventionelle Energie, nach Ländern",mapbox_style="open-street-map",margin={"r":0,"l":0,"b":0})
fig.show()

Die Karte kann man nach Ländern filtern und sich einzelne Punkte oder Länder ansehen.

Alternativ kann man die Karte farblich statt nach Ländern auch nach Kraftwerktyp visualisieren:

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(joined, lat="lat", lon="lon", color='Production Type',hover_name="Name",hover_data=["capacity",'Production Type'],zoom=3)
fig.update_layout(title="Kraftwerkskarte, konventionelle Energie, nach Production Type",mapbox_style="carto-positron",margin={"r":0,"l":0,"b":0})
fig.show()

In [ ]:
# areas containing load content
def createValidAreas():
    valid_areas=[]
    for e in Area:
        country_code = e.name
        try:
            load[country_code] = client.query_load(country_code, start=start,end=end)
            valid_areas.append(country_code)
        except Exception as e:
            pass
            #print('failed:',country_code, e)
    valid_countries = list(filter(lambda x: len(x)<=2,valid_areas))
    return valid_areas, valid_countries

# takes ~40s
%time valid_areas, valid_countries = createValidAreas()

In [ ]:
print('Countries:',"['"+"','".join(valid_countries)+"']")

In [ ]:
import plotly.graph_objects as go
data= []
valid_countries=['DE','BE','NL','FR','CZ','PL','NO','IT','ES']
#for country_code in valid_countries:
for country_code in ['DE','BE','NL','FR','CZ','PL','NO','IT','ES']:
    load[country_code]=client.query_load(country_code, start=start,end=end)
    data.append(go.Scatter(x=load.index,y=load[country_code]/1000,name=country_code,   
        fill='tozeroy',
        fillcolor='rgba(26,0,65,0.1)',
        #stackgroup='one',
        #stackgaps='interpolate'
        ))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
all_data= data
textbox = widgets.Dropdown(
    description='Länder:   ',
    value='Alle',
    options=valid_countries+['Alle'],
    multiselect=True,
)

fig = go.FigureWidget(data=data)

fig.update_layout(
    title="Netzlast der Länder",
    xaxis_title="Zeit",
    yaxis_title="Leistung in MW",
    legend_title="Länder",
    font=dict(
        #family="Courier New, monospace",
        #size=18,
        #color="RebeccaPurple"
    ),)

import time
def response(change):
    with fig.batch_update():
        fig.data = []        
        if textbox.value=='Alle':
            for country_code in ['DE','BE','NL','FR','CZ','PL','NO','IT','ES']:
                load[country_code]=client.query_load(country_code, start=start,end=end)
                fig.add_trace(go.Scatter(x=load.index,y=load[country_code]/1000,name=country_code,   
                    fill='tozeroy',
                    fillcolor='rgba(26,0,65,0.1)',
                    #stackgroup='one',
                    #stackgaps='interpolate'
                    ))
        else:
            fig.add_trace(go.Scatter(x=load.index,y=load[textbox.value]/1000,name=textbox.value,
                fill='tozeroy',
                fillcolor='rgba(26,0,65,0.1)',
                #stackgroup='one',
                #stackgaps='interpolate'
                ))
        fig.update_layout(showlegend=True)
textbox.observe(response, names="value")

#fig.show()
widgets.VBox([widgets.HBox([textbox]),
              fig])

Die Verläufe sind in ihrer Bewegung der Netzauslastung sehr ähnlich. Je größer das Land, desto größer sind auch die Schwankungen.

Wenn man sich die Daten anguckt, sieht man, dass nicht alle Daten viertelstündlich, wie DE vorliegen. Spanien (ES) melde die Daten nur stündlich. Das muss bei der Speicherung beachtet werden.

Betrachten wir noch die Kapazitäten der Länder:

## Kapazität der Länder nach Jahren

In [ ]:
start1 = pd.Timestamp('20150101', tz='Europe/Berlin')
end1 = pd.Timestamp('20200102', tz='Europe/Berlin')
def getCapacities():
    capacity = {}
    for country_code in valid_countries:
        capacity[country_code] = client.query_installed_generation_capacity(country_code, start=start1,end=end1)
    return capacity

%time capacity = getCapacities()

In [ ]:
from ipywidgets import widgets

country_code =list(capacity.keys())[0]

textbox = widgets.Dropdown(
    description='Länder:   ',
    value=country_code,
    options=list(capacity.keys()),
)
data= []
curDat = capacity[country_code]
curDat.fillna(0,inplace=True)
for fuel in list(curDat.keys()):
        data.append(go.Bar(x=curDat.index,y=curDat[fuel]/1000,name=fuel,   
    #fill='tozeroy',
    #stackgroup='one',
    #stackgaps='interpolate'
    ))

fig = go.FigureWidget(data=data)
fig.update_layout(
    title="Erzeugungskapazitäten der Länder, unterteilt nach Sorte",
    xaxis_title="Jahr",
    yaxis_title="Erzeugungskapazität in MW",
    legend_title="Sorten",
    barmode='stack',
    font=dict(
        #family="Courier New, monospace",
        #size=18,
        #color="RebeccaPurple"
    ),)

def response(change):
    with fig.batch_update():
        fig.data = []
        curDat = capacity[textbox.value]
        curDat.fillna(0,inplace=True)
        for fuel in list(curDat.keys()):
            fig.add_trace(go.Bar(x=curDat.index,y=curDat[fuel]/1000,name=fuel,                   
                ))
        fig.update_layout(showlegend=True,barmode='stack')
textbox.observe(response, names="value")

#fig.show()

widgets.VBox([widgets.HBox([textbox]),
              fig])

Hier erkennt man sehr gut den jeweiligen Energiemix der Länder.

Frankreich setzt sehr auf Kernenergie. DE hat einen sehr gemischten Energiemix.

Italien hat 2015 und 2016 "Other", Datenlage wurde jedoch aufgebessert

Hier hat man leider auch nicht immer die selben verfügbaren Werte. Zur Speicherung in einer Datenbank muss man also das Schema dynamisch aufbauen oder vorher alle jemals verfügbaren Erzeugnisformen kennen.

Später möchte man in einer Visualisierung den einzelnen Erzeugnissen auch immer die selbe Farbe geben (Solar=gelb,Kohle=Schwarz usw), weshalb man auch hier mit fehlenden oder hinzukommenden Werten klar kommen muss.  

Holprig wird es erst bei den Stromerzeugungs-Daten:

## Stromerzeugung in Deutschland in aktueller Woche

In [ ]:
def replaceStr(string):
    '''
    Apache Spark gefallen einige Zeichen der Columns nicht.
    Diese werden hier ersetzt
    '''
    
    st = str.replace(string,')','')
    st = str.replace(st,'(','')
    st = str.replace(st,',','')
    st = str.replace(st,"'",'')
    st = st.strip()
    st = str.replace(st,' ','_')
    return st

Zunächst holen wir uns die Daten über die API:

In [ ]:
#gen=pd.DataFrame() # does not work with differing columns
gen={}
country='GR'
gen[country] = client.query_generation(country, start=start,end=end) # no data for DE
gen[country]['time']=gen[country].index
country='DE'
gen[country] = client.query_generation(country, start=start,end=end) # no data for DE
gen[country]['time']=gen[country].index

In [ ]:
gen['DE'].head()

In [ ]:
gen['GR'].head()

Man sieht also, dass die Daten unterschiedliche Strukturen haben. Nicht immer erfolgt die Aufschlüsselung nach Erzeugnissen (Actual Aggregated) und der zur Erzeugung benötigten Energie (actual Consumption)

Hierzu müssen die Daten über die Differenzen gebildet werden:

In [ ]:
#convert multiindex to single index
gen['DE'].columns = list(map(replaceStr, map(str,gen['DE'].columns)))
# unpivot data for visualization
g = gen['DE'].melt(id_vars=['time'],  var_name='kind', value_name='value')

In [ ]:
import plotly.express as px 
fig = px.line(g, x='time', y='value', color='kind',title='Stromerzeugnis in Deutschland der letzten 5 Tage in kWh')

fig.show()

In dieser Grafik kann man mit einem Doppelklick auf "Hydro Pumped Storage Actual Aggregated" und anschließendem einfach Klick "Hydro Pumped Storage Actual Consumption" sich zwei Series zusammen angucken.

Hier sieht man, dass der Pumpwasserspeicher tagsüber Strom erhält und nachts Strom in das Netz einspeist.


Für Griechenland erhält man diese Daten nicht aufgeschlüsselt, weshalb hier nur der eingespeiste Wert verfügbar ist.

In [ ]:
#convert multiindex to single index
gen['GR'].columns = list(map(replaceStr, map(str,gen['GR'].columns)))
# unpivot data for visualization
g = gen['GR'].melt(id_vars=['time'],  var_name='kind', value_name='value')

In [ ]:
import plotly.express as px 
fig = px.line(g, x='time', y='value', color='kind')
fig.update_layout(
    title='Stromerzeugnis in Griechenland der letzten 5 Tage in kWh',
    xaxis_title="Datum",
    yaxis_title="Erzeugnis in kWh",
    legend_title="Erzeugungsform",    
)
fig.show()

Damit man diese Daten nun unter einen Hut bekommt, muss man nun jeweils die Differenz zwischen Consumption und Aggregation erhalten und den MultiIndex gegebenenfalls umbenennen.

In [ ]:
def calcDiff(data):
    '''
    Berechnet jeweils Differenzen zwischen den zugehörigen zwei Spalten.
    '''
    dat=data
    for c in filter(lambda x:x.endswith('_actual_aggregated'), dat.columns):        
        new = str.replace(c,'_actual_aggregated','')
        dif = list(filter(lambda x: x.endswith('_actual_consumption') and x.startswith(new), dat.columns ))
        if len(dif) > 0:
            # wenn es beides gibt wird die Differenz gebildet
            print(dif[0])
            dat[new]=dat[c]-dat[dif[0]]
            del dat[c]
            del dat[dif[0]]
        else:
            # sonst wird direkt 
            dat[new]=dat[c]
            del dat[c]
    for c in filter(lambda x:x.endswith('_actual_consumption'), dat.columns):
        # wenn es nur Verbrauch aber kein Erzeugnis gibt, mach negativ
        new = str.replace(c,'_actual_consumption','')
        dat[new]=-dat[c]
        del dat[c]
    return dat

In [ ]:
data = gen['DE']

In [ ]:
data.columns = list(map(replaceStr, map(str,data.columns)))
data.fillna(0, inplace=True)
# calculate difference betweeen agg and consumption
data=calcDiff(data)

Nun haben wir einheitliche Namen der Erzeugnisformen:

In [ ]:
list(data.columns)[1:]

In Deutschland erhalten wir nun die Netto Erzeugnisse. Das erzeugt für den Pumpspeicher negative Werte, was die Interpretation auch erleichtert.

Im Area-Plot ist die Visualisierung trotzdem korrekt und stellt die Erzeugung nach Erzeugungsform dar.

In [ ]:
g = data.melt(id_vars=['time'],  var_name='kind', value_name='value')
fig = px.area(g, x='time', y='value', color='kind')
fig.update_layout(
    title='Stromerzeugnis in Deutschland der letzten 5 Tage in kWh',
    xaxis_title="Datum",
    hovermode="closest",
    yaxis_title="Erzeugnis in kWh",
    legend_title="Erzeugungsform",    
)
fig.show()

# Speicherung der Daten

Bisher haben wir alle Analysen auf den Daten der API gemacht.
Um Analysen über die Historie durchzuführen, müssen die Daten paketweise von der API runtergeladen werden und in einem sinnvollen Format abgespeichert werden.

Nach einem ersten Ansatz, welcher pro Land eine eigene Tabelle vorsah, wurde nun das folgende Datenbank-Schema gewählt:

![ER-Diagramm](diagrams-ENTSO-E.png)

Hier gibt es nur noch für die Kraftwerks-Erzeugungen eine eigene Tabelle pro Land.
Wobei diese vermutlich auch sinnvoller als unpivot Tabelle behandelt werden sollte.

Es ist möglich die Daten in Parquet zu speichern.
Ein andere sinnvolle Möglichkeit erscheint SQLite zu sein, da man es recht leicht ansprechen kann und dank in-memory-Datenbanken auch ordentlich Performance in einem nicht-verteilten System erhält.
Da lediglich viertelstündlich Daten gespeichert werden und die Datenmenge ungefähr 60 Länder * ~ 16 Spalten + 60 Länder * ~50 Kraftwerke also ganz grob 3600 Werte pro Viertelstunde enthält, welche in einem relationalen Format gespeichert werden, kommt man hier mit konventionellen RDBMS gut davon.

Zum Herunterladen der Daten wurde eine Klasse angelegt, welche die Paketierung übernimmt und sowohl die Speicherung in eine SQLite-Tabelle übernimmt, wie auch in Parquet übernimmt.

Hierbei wurde viel mit der Speicherung der Datumswerte und einer Partitionierung der Parquet-Dateien experimentiert, welche Performance-technisch auf dem Programmier-Laptop nichts änderten

Die Filtern der Zeitwerte läuft nativ schon sehr gut, da der String-Wert in der SQLite-Datenbank standardmäßig als Datum interpretiert wird.

Der TimeStamp lässt sich in SQLite zur Gruppierung mit der SQLite-Funktion strftime manipulieren.
Durch `strftime("%Y-%m-%d %H:00:00", "index") as time` erhält man den zur Stunde abgeschnittenen Wert.
Mit `strftime("%Y-%m-%d", "index") as time` respektive den Wert für ein Zusammenfassen nach Tagen

Eine Gruppierung nach `time` und aggregation mit avg bringt die gewünschten Ergebnisse.

Mit Apache Spark und Parquet läuft das ziemlich analog.

Den TimeStamp kann man sich über die spark.sql.function date_trunc abschneiden lassen. Hier kann man direkt sagen, wo abgeschnitten werden soll: `spark_df.withColumn("time", date_trunc('day',"time"))`

## Einschub SPARK

Der Einfachheithalber wurden die Daten bereits herunter geladen und mithilfe von Parquet abgespeichert

In [ ]:
import findspark

findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

conf = SparkConf().setAppName('entsoe').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
loadDE = spark.read.parquet('data/spark/DE/query_load')

In [ ]:
beg=loadDE.select('time').rdd.min()
end=loadDE.select('time').rdd.max()

print('Datensequenz von {} bis {}'.format(beg[0],end[0]))

In [ ]:
from pyspark.sql.functions import date_trunc
groupTime='month'

loadBE = spark.read.parquet('data/spark/BE/query_load').withColumn(groupTime, date_trunc(groupTime,"time"))
floadBE = spark.read.parquet('data/spark/BE/query_load_forecast').withColumn(groupTime, date_trunc(groupTime,"time"))

In [ ]:
%time pl=loadBE.groupby(groupTime).avg('0').toPandas()
%time pf=floadBE.groupby(groupTime).avg('0').toPandas()

In [ ]:
import plotly.express as px


pl.sort_values(groupTime,inplace=True)
pf.sort_values(groupTime,inplace=True)
pf['forecast']=pf['avg(0)']
pf['actual']=pl['avg(0)']
fig = px.line(pf, x=groupTime, y=['actual','forecast'], title='Auslastung Belgien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kW gemittelt pro Stunde')
fig.show()

Zum Vergleich das ganze auch in SQLite:

In [ ]:
import sqlite3
from contextlib import closing
country = 'BE'
selectString='strftime("%Y-%m-01", "index") as time, avg("0") as value'
groupString='strftime("%Y-%m-01", "time")'
with closing(sqlite3.connect('data/entsoe.db')) as conn:
    query = f"select {selectString} from query_load where country='{country}' group by {groupString}"
    %time load = pd.read_sql_query(query,conn,index_col='time')
    query = f"select {selectString} from query_load_forecast where country='{country}' group by {groupString}"
    print(query)
    %time forecast = pd.read_sql_query(query,conn,index_col='time')

In [ ]:
load['forecast']=forecast['value']

fig = px.line(load, x=load.index, y=['value','forecast'], title='Auslastung Belgien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kW gemittelt pro Stunde')
fig.show()

Man sieht hierbei, dass die SQLite-Abfrage 5-10 mal schneller ist als Apache Spark mit Parquet.

Das liegt zum einen an der mit 5 GB Daten noch recht geringen Datenmenge, zum anderen daran, dass die Daten ordentlich gefiltert werden können.

Einene Performance-Test auf dem Spark-Cluster hab ich mangels Zeit nicht durchgeführt.

# Analysen auf Basis der gesamten Historiendaten

Da nun alle Historiendaten lokal verfügbar sind, können spannende Fragestellungen beantwortet werden. Auf einige ausgewählte möchte ich nachfolgend eingehen

## Wie hat sich die Stromförderung durch Corona verändert?

Spannend wären bspw Abweichungen von den Vormonaten.

In [ ]:
from entsoe_sqlite_manager import EntsoeSQLite, EntsoePlantSQLite, Filter
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, date, timedelta
import pandas as pd

country = 'DE'
dm = EntsoeSQLite('data/entsoe.db')
pdm = EntsoePlantSQLite('data/entsoe-plant.db')
filt = Filter(datetime(2015, 3, 2), datetime(2020, 12, 2), 'month')

### In Deutschland

In [ ]:
load_germany_corona = dm.load(country,filt)
fig = px.line(load_germany_corona, title='Netzlast Deutschland pro Monat seit 2015')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Netzlast in kW')
# uncomment this to get a view on the real dimensions
#fig.update_yaxes(rangemode="tozero")
fig.show()

Man kann hier deutlich erkennen, dass es vorher noch nie einen so starken Einbruch des Energiebedarfs gegeben hat wie zu Beginn der Pandemie 2020.

Außerdem sieht man, dass im Winter viel geheizt wird und die Netzlast dadurch im Winter viel höher ist.

### Beispielsweise am Kraftwerk Weisweiler

In [ ]:
plants = ['Weisweiler E','Weisweiler F','Weisweiler G','Weisweiler H']
plantDat= pdm.plantGen(plants,filt).pivot(columns='name', values='value')

In [ ]:
fig = px.line(plantDat, title='Netzlast Deutschland pro Monat seit 2015')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Netzlast in kW')
fig.show()

In Weisweiler hat Sektor F schon Ende 2019 eine längere Wartung. Alle anderen Sektoren wurden zu Beginn der Pandemie deutlich runter gefahren.

## Fluss zu benachbarten Ländern


* Karte mit Änderungsflüssen

Es kann festgestellt werden wie der Fluss zu benachbarten Ländern ist.

Hierzu kann beim Anklicken eines Landes der Netto-Transfer (Exportiert-importiert) berechnet werden.
Die Daten müssen hierzu in ein geeignetes Format gebracht werden (neighbours Tabelle an crossboarders joinen..)
Eigentlich müsste an jede Beziehung der Neighbours-Tabelle eine Series gejoined werden ->
    realisieren durch crossboarder-Tabelle, welche alle nachbarschaftsbeziehungen als Key enthält

Es gibt also eine Tabelle, welche für jede Beziehung zwischen benachbarten Ländern eine Spalte der Form from-to also beispielsweise FR-BE enthält.

Nun muss man die Spalten filtern, welche vor dem - das gewünschte Kürzel enthält. (Export von Kürzel zu den Nachbarn)
Anschließend kann man zu jeder Spalte die gedrehte Reihenfolge nehmen und erhält den Import eines Nachbarn.
Somit kann man durch eine Differenz direkt den Netto-Fluss von der Datenbank abfragen.

Für die SQLite Datenbank sieht das beispielswiese so aus:

In [ ]:
def _selectBuilder(neighbours):
    res = ''
    for x in neighbours:
        fr = x.split('-')[0]
        to = x.split('-')[1]
        # export - import
        res += f'avg("{fr}-{to}"-"{to}-{fr}") as diff_{to}'
        res += ','
    return res

def _neighbours(fromC):
    with closing(sqlite3.connect('data/entsoe.db')) as conn:
        query = 'select * from query_crossborder_flows where 0=1'
        columns = pd.read_sql_query(query, conn).columns
    nei = []
    for x in columns:
        sp = x.split('-')
        if sp[0] == fromC:
            nei.append(x)
            # nei.append(sp[1]+'.'+sp[0])
    return nei

In [ ]:
country = 'ES'

with closing(sqlite3.connect('data/entsoe.db')) as conn:
    nei = _neighbours(country)
    n = _selectBuilder(nei)
    selectString='strftime("%Y-%m-%d %H:00:00", "index") as time,'
    groupString='strftime("%Y-%m-%d  %H:00:00", "index")'
    
    query = f'select {selectString}{n} "index" from query_crossborder_flows group by {groupString}'
    print(query)
    neigh = pd.read_sql(query,conn,index_col='index')

In [ ]:
fig = px.line(neigh, x=neigh.index, y=['diff_FR','diff_PT'], title='Netto Export an Nachbarn von Spanien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Export - Import an Nachbarn')
fig.show()

**durch ziehen eines Intervalls, kann man den Betrachtungsraum verkleinern. Ein Doppelklick in das Diagramm springt wieder in den Ursprungsbereich**

Zu erkennen ist, dass Spanien von Portugal importiert und mehr nach Frankreich exportiert.

Wählt man ein Intervall durch klicken und ziehen aus, so stellt man ebenso fest, dass grade in der Nacht von Frankreich importiert wird, während am Tag nach Frankreich exportiert wird.

Von Portugal wird Strom tagsüber eingekauft (vermutlich an Frankreich weiter verkauft), während nachts Strom (vermutlich von Frankreich) nach Portugal verkauft wird.

Das hängt damit zusammen, dass Portugal auf Windenergie setzt und zu milden Zeiten den Kernenergie-Strom aus Frankreich bezieht.

## Visualisierung von Umweltmetriken

Eine weitere Anforderung war die Visualisierung von Umweltmetriken. Hierzu wird die Erzeugung mit einem Vektor multipliziert. Die genaue Beschreibung der Berechnung, sowie das anlegen einer Chloropleth-Karte wird nachfolgend beschrieben.

Die Umweltmetriken lassen sich anschließend im finalen Dashboard betrachten.

### Umweltbelastung aus den Erzeugungswerten berechnen

Die Daten für die Umweltbellastung stehen am NOWUM-Energy Institut in g/kWh zur Verfügung.
Mithilfe dieser kann durch Multiplizieren der Generationswerte in kWh/h ein Belastungswert in g/h berechnet werden.

Diese Möglichkeit wurde im Dashboard durch Multiplizieren des DataFrames mit einer Pandas Series implementiert.

So wurde eine interaktive Karte erzeugt, welche pro Land den gewichteten Belastungswert als durchschnittliche Belastung pro erzeugte kWh berechnet.

Dieser berechnet sich durch: 

$P_{energieform}$ = die Erzeugungskapazität der Energieform in MWh\
$P_{Gesamt}$ = Gesamterzeugungskapazität in MWh\
$C_{metric,energieform}$ = Wert der gewählte Klimametrik pro Energieform

$$ \frac{\sum_{Erzeugungsarten}{C_{metric}*P_{energieform}}}{P_{Gesamt}}$$

Dabei wird die Annahme getroffen, dass die Erzeugungskapazitäten der verschiedenen Energieformen gleichmäßig ausgelastet werden.

Für eine feinere Berechnung der Metriken lässt sich ebenfalls statt der Verstromten Energie nach Energieform auch die hierzu benötigte Menge $CO_2$ und andere Berechnen.

Als Klimakennzahlen stehen hierbei folgende zur Verfügung:

* $CO_2$ ohne Vorkette (VK)
* $CO_2$ mit Vorkette (VK)
* Summe $NO_X$ Wert
* Summe Staub
* Eigenverbrauch in kWh/erzeugte MWh

### Chloropleth-Karte zur Visualisierung von Ländern

Die Umrisse von Europäischen Ländern wurden von https://geojson-maps.ash.ms/ heruntergeladen.

Diese Visualisierung wird im finalen Dashboard für die Klima-Metriken verwendet.

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
countries = ['DE','FR','GR','PL','LU','IT']

with open("europe.geo.json", "r", encoding="utf-8") as f:
    geo = json.load(f)    

#geo['features'][0]
df = pd.DataFrame()
df['countries']=countries
df['values']=list(map(lambda x: ord(x[0]),countries))

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=geo, locations="countries", color='values',
                           color_continuous_scale="Jet",
                           featureidkey="properties.iso_a2",
                           range_color=(65, 80),
                           mapbox_style="carto-positron", # open-street-map
                           zoom=3, center = {"lat": 50.0902, "lon": 10.7129},
                           opacity=0.5,
                           labels={'values':'Werte'}
                          )
fig.update_layout(title='Beispielkarte mit Kunstwerten für ausgewählte Länder')
fig.show()

# Design-Änderung für interaktives Dashboard

Die obigen Grafiken sind zwar schon sehr schön, für den täglichen Gebrauch möchte man allerdings ähnlich wie https://energy-charts.info/ die Möglichkeit haben das Zeitintervall und den Betrachtungsraum zusammen zu filtern.

Deshalb kam die Idee, sich mit hilfe der Software-Bibliothek Dash, welche auf den oben verwendeten Komponenten der Visualisierungssoftware Plotly aufbaut, ein Dashboard zu basteln.

Hierzu wurde zunächst überlegt, welche Daten im Dashboard sichtbar sein sollen.
Deshalb wurde (auch wenn es in Python unüblich ist) ein Interface für einen DataManager angelegt:

In [ ]:
import inspect
from entsoe_data_manager import *
lines = inspect.getsource(EntsoeDataManager)
print(lines)

lines = inspect.getsource(EntsoePlantDataManager)
print(lines)

Hier wird in einigen Fällen ein Filter übergeben. Dieser enthält ein begin, ende und eine Gruppierung, also eine Feinheit der Daten.

In [ ]:
lines = inspect.getsource(Filter)
print(lines)

Der DataManager wurde nun einmal für SQLite und einmal für Apache Parquet implementiert und ausprobiert.

Mit Dash wurde anschließend um den DataManager ein Dashboard gebaut, welches die Daten visualisiert und Interaktionen ermöglicht.

Dieses Dashboard findet sich hier: 

https://nicht.datensch.eu

oder aus dem FH-Aachen-VPN:

https://service-fb9.fh-aachen.de/energy/entsoe

# Analysen mithilfe des Data-Managers
Der DataManager stellt alle wichtigen statischen Daten, wie auch die täglichen Werte (Generation und Load) zur Verfügung.

Dieser deutlich bequemere Abfrage wird nun genutzt um Datenanalysen durchzuführen

## Kapazitäten-Vergleich eines Landes mit der Summe zugehöriger Kraftwerke

In der Historie haben wir pro Jahr die Kapazitäten nach Produktionsart zur Verfügung.

Außerdem sind die Kapazitäten der Kraftwerke jedes Landes bekannt.

So müsste die Summe der Kraftwerks-Kapazitäten der Gesamtsumme des Landes entsprechen. Das wird nachfolgend geprüft.

In [ ]:
from entsoe_sqlite_manager import EntsoeSQLite, EntsoePlantSQLite, Filter
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, date, timedelta
import pandas as pd

In [ ]:
country = 'NL'
dm = EntsoeSQLite('data/entsoe.db')
pdm = EntsoePlantSQLite('data/entsoe-plant.db')
filt = Filter(datetime(2020, 8, 1), datetime(2020, 9, 2), 'hour')

In [ ]:
countries = dm.countries()
countries

In [ ]:
# Klimadaten in g pro generierter kWh
climate = dm.climateImpact()
climate

Nun wollen wir einmal die Kapazitätsdaten der Länder mit der SQLite-Abstraktion analysieren:

In [ ]:
def capacities(country):
    perCap1 = pdm.capacityPerPlant(country)
    perCap1['capacity'] = perCap1['capacity'].astype(float)
    perCap = perCap1.groupby('Production_Type').sum()['capacity']

    cap = dm.capacity(country)
    del cap['country']
    c = cap.T

    c['2014']=perCap
    c = c.T
    c.fillna(0,inplace=True)
    # 2014 is per capacity. stupid hack
    return c

Hier sieht man je nach Land deutliche Abweichungen zwischen echten Daten der Jahre und der Summe der Kapazitäten der einzelnen Kraftwerke.

Das hängt damit zusammen, dass ehemalige Kraftwerke auch in den Kraftwerkslisten vorhanden sind.
Dagegen fehlen andere Verstromungs-Kraftwerke komplett.

Die fehlenden Daten unterscheiden sich von Land zu Land sehr.
So hat Portugal (PT) sehr schlecht gepflegte Kraftwerksdaten.
Bei Spanien sieht es gut aus.

Deutschland kann hier nicht evaluiert werden, da es in DE verschiedene Marktgebiete gibt.

**Damit die Summe der Kraftwerksdaten (perPlant Kapazität) im selben Diagramm visualisiert wird, wie die Landes-Kapazitäten nach Jahr, wurde ein fiktiver Balken für 2014 verwendet. Die Jahresdaten existieren erst ab 2015**

In [ ]:
country = 'FR'
# 2014 is sum of per plant data
px.bar(capacities(country)/1000,labels={
                     "value": "Erzeugungskapazität in GW"
                 },title="Erzeugungskapazitäten "+country+"; 2014 ist perPlant Kapazität")

In [ ]:
# für Deutschland sehr kompliziert/uneindeutig
perCap1 = pdm.capacityPerPlant('DE_50HZ')
perCap2 = pdm.capacityPerPlant('DE_AMPRION')
perCap3 = pdm.capacityPerPlant('DE_LU')
perCap = pd.concat([perCap1,perCap2,perCap3])
perCap = pdm.capacityPerPlant('DE_AT_LU')
perCap['capacity'] = perCap['capacity'].astype(float)
perCap = perCap.groupby('Production_Type').sum()['capacity']

cap = dm.capacity('DE')
del cap['country']
c = cap.T

c['2014']=perCap
c = c.T
c.fillna(0,inplace=True)
# 2014 is per capacity. stupid hack
px.bar(c/1000,labels={
                     "value": "Erzeugungskapazität in GW"
                 },title="Erzeugungskapazitäten "+country+"; 2014 ist perPlant Kapazität")

## Ist Generation der Country = Generation der zugehörigen Plants?

Die Erzeugnisse der einzelnen Länder müssten theoretisch der Summe aller Kraftwerke des Landes entsprechen. Das wird hier überprüft

So kann man sich die aktuellen Erzeugnisse der einzelnen Kraftwerke vom PlantDataManager holen:

In [ ]:
filt = Filter(datetime(2019, 9, 1), datetime(2020, 9, 2), 'hour')
names = pdm.getNames()

plantDat = pd.DataFrame()
plants = ['GTHKW Nossener Bruecke','DOEL 2', 'TIHANGE 2', 'TIHANGE 1']
#for n in plants:
#    plantDat[n] = pdm.plantGen([n], filt)['value']
plantDat= pdm.plantGen(plants,filt).pivot(columns='name', values='value')

plantDat.fillna(0,inplace=True)

px.line(plantDat,labels={
                     "value": "durchschnittlich eingespeiste Energie in kW",
                 },title="Generation per plant")

In [ ]:
country='ES'
filt = Filter(datetime(2020, 3, 1), datetime(2020, 9, 2), 'hour')

plants =list(names[names['country']==country]['name'])
genDat = pdm.plantGen(plants,filt)
sumData= genDat['value'].groupby('time').sum()

data = pd.DataFrame(sumData)
data['Generation per plant'] = dm.generation(country,filt).sum(axis=1)
data.fillna(0,inplace=True)

px.line(data,labels={"value": "Generation sum", "_value": 'Energie in kW'
                 },title="Generation sum vs per Plant for "+country)

Hier fehlen unmengen an Daten.
Vermutlich liegt das an Kleinerzeugern, erneuerbaren Energien (Windräder) und sonstigen schlechten Daten.

Gehen wir der Sache also auf den Grund:

In [ ]:
country='FR'
filt = Filter(datetime(2020, 3, 1), datetime(2020, 9, 2), 'day')

plants =list(names[names['country']==country]['name'])
genDat = pdm.plantGen(plants,filt)
sumData= genDat.groupby(['time','type']).sum('value')
gen = dm.generation(country,filt)
plantSum =sumData.unstack()
plantSum.columns = plantSum.columns.droplevel(0)

In [ ]:
plantSum['Hydro Pumped Storage']=-plantSum['Hydro Pumped Storage']
for c in  list(plantSum.columns):
    d = pd.concat([plantSum[c],gen[c]],axis=1,keys=["PlantData "+c, "Generation: "+c])
    d.plot(rot=45)

Der Unterschied entsteht in Spanien also hauptsächlich durch die Daten in der Solar- und Wasserkraft.

Andere Länder (bspw FR) haben jedoch deutlich schlechtere Datenstände

## Passt Erzeugung+Import-Export=Load?

In [ ]:
country = 'ES'
filt = Filter(datetime(2019, 1, 1), datetime(2020, 2, 2), 'day')

neighbours = dm.crossborderFlows(country, filt)
load = dm.load(country, filt)
generation = dm.generation(country, filt)

gen = generation.sum(axis=1)
ne = neighbours.sum(axis=1)
load['generation']=gen-ne
px.line(load,labels={
                     "value": "Load in kW",
                     "generation": "Generation and Transfer", "_value": 'Energie in kW'
                 },title="Load = Generation+Import-Export for "+country)  

Das passt perfekt. Die Daten sind sinnvoll

Alles weitere findet sich dann im großen Dashboard:

https://nicht.datensch.eu/entsoe

oder aus dem FH-Aachen-VPN:

https://service-fb9.fh-aachen.de/energy/entsoe


# Ausblick

Nachfolgend noch ein paar Ideen, deren Umsetzung im Rahmen des zeitlichen Aufwands des Moduls nicht möglich war.

## Animation über die Zeit, mit Darstellung der Transfermenge durch die Dicke der Linie

Animations-Bibliothek, welche einen Graph auf einer OSM-Karte mit der Zeit animiert?
Daten stehen zur Verfügung

## Sankey-Diagramm über Gesamt-Generierte Daten und Flüsse zwischen den Ländern

Über die Jahre kann man dann eine Art Zustandsdiagramm machen, welcher Staat wie autark ist und wer stets Strom zugeführt bekommen muss.

Nach der obigen Erkenntnis, ist eine solche Abhängigkeit allerdings wenig aussagend, da die Export-Richtung oft für Tag eine andere ist als für die Nacht.

Energy-charts.info versucht so etwas. Visualisiert den restlichen Export/Usage der Nachbar-Länder nicht.
https://energy-charts.info/charts/import_export/chart.htm

## Performance-Test des Spark-Clusters 

Durch einer kurzen Testreihe mit einem lokalen Spark-Client wurde der Einsatz von SQLite begründet. Durch die Verteilung der Datenverarbeitung auf mehrere Knoten, könnte hierbei eventuell eine bessere Performance erlangt werden.

Da die Daten bereits in einer sehr guten Form vorhanden sind und SQLite schon wahnsinnig schnell läuft, halte ich große Verbesserungen für sehr unwahrscheinlich

## Wie gut ist die Vorhersage der erzeugten Daten? Können wir etwas besseres?

ENTSO-E schreibt:
> The day-ahead forecast is calculated (estimated) on the historic load profile on similar days, taking into account the variables that affect electricity demand, such as weather conditions, climate and socioeconomic factors.

Vielleicht lässt sich mit Machine-Learning ein besseres Modell finden.

Problematik: zyklische Trainingsdaten, weniger Daten (Wetter, Klima und Sozioökonomische Daten fehlen)

Abweichung der Schätzungen vergleichen